In [11]:
import random
import speech_recognition as sr

# Lists of random words to construct sentences
subjects = ["the cat", "a dog", "a bird", "the man", "a woman", "the child", "an elephant", "the teacher", "the student", "a programmer"]
verbs = ["eats", "jumps over", "runs", "walks", "sits on", "stands by", "talks to", "reads", "writes", "plays with"]
objects = ["a book", "the grass", "a tree", "a computer", "a ball", "the food", "a chair", "a game", "the phone", "a car"]
additional_phrases = ["quickly", "happily", "sadly", "angrily", "excitedly", "silently", "loudly", "carefully", "eagerly", "curiously"]

def generate_random_sentence(num_words=10):
    sentence = []
    for _ in range(num_words // 4):  # Each sentence will have approximately subject-verb-object-adverb structure
        subject = random.choice(subjects)
        verb = random.choice(verbs)
        obj = random.choice(objects)
        additional = random.choice(additional_phrases)
        sentence.extend([subject, verb, obj, additional])
    return ' '.join(sentence[:num_words]).lower()

def compare_sentences(random_sentence, user_sentence):
    if random_sentence == user_sentence:
        return True, None
    else:
        mistakes = []
        random_words = random_sentence.split()
        user_words = user_sentence.split()
        min_length = min(len(random_words), len(user_words))
        
        for i in range(min_length):
            if random_words[i] != user_words[i]:
                mistakes.append((i, random_words[i], user_words[i]))
        
        if len(random_words) > len(user_words):
            for i in range(min_length, len(random_words)):
                mistakes.append((i, random_words[i], None))
        elif len(user_words) > len(random_words):
            for i in range(min_length, len(user_words)):
                mistakes.append((i, None, user_words[i]))
        
        return False, mistakes

def recognize_speech_from_mic(recognizer, microphone):
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        print("Listening...")
        audio = recognizer.listen(source)

    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    try:
        response["transcription"] = recognizer.recognize_google(audio).lower()
    except sr.RequestError:
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        response["success"] = False
        response["error"] = "Unable to recognize speech"

    return response

def main():
    random_sentence = generate_random_sentence(num_words=10)
    print(f"Repeat this sentence: {random_sentence}")

    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    response = recognize_speech_from_mic(recognizer, microphone)

    if response["success"]:
        user_sentence = response["transcription"]
        print(f"You said: {user_sentence}")

        result, mistakes = compare_sentences(random_sentence, user_sentence)
        if result:
            print("You did it! The sentences match.")
        else:
            print("The sentences do not match. Here are the mistakes:")
            for index, correct_word, user_word in mistakes:
                if correct_word and user_word:
                    print(f"Word {index + 1}: expected '{correct_word}' but got '{user_word}'")
                elif correct_word:
                    print(f"Word {index + 1}: expected '{correct_word}' but got nothing")
                else:
                    print(f"Word {index + 1}: got extra word '{user_word}'")
    else:
        print("I didn't catch that. What did you say?")
        if response["error"]:
            print(f"ERROR: {response['error']}")

if __name__ == "__main__":
    main()


Repeat this sentence: the student jumps over a tree angrily the student walks a ball eagerly
Listening...
You said: the student jumps over a tree angrily the student walks about eagerly
The sentences do not match. Here are the mistakes:
Word 11: expected 'a' but got 'about'
Word 12: expected 'ball' but got 'eagerly'
Word 13: expected 'eagerly' but got nothing
